In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
!pip install --upgrade transformers
!pip install torch torchvision transformers opencv-python pandas tqdm scikit-learn


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install --upgrade jupyter
!pip install --upgrade ipywidgets


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import cv2
import pandas as pd
import torch
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
from tqdm import tqdm
from sklearn.model_selection import train_test_split


In [5]:
"""
base_video_path = '/content/drive/MyDrive/aip/MovieData/'

# Load CSV file

csv_file = '/content/drive/MyDrive/aip/MovieData/fixed_downloaded_videos.csv'
df = pd.read_csv(csv_file)
print(df.head())
"""
# Set the new base video path

# Load CSV file with the new path
csv_file = r'fixed_downloaded_videos.csv'
df = pd.read_csv(csv_file)

# Print the dataframe to check if it's loaded correctly
print(df.head())

     video_id                  video_path  \
0    21179416    videos_1000\21179416.mp4   
1     5629184     videos_1000\5629184.mp4   
2  1063125190  videos_1000\1063125190.mp4   
3  1039695998  videos_1000\1039695998.mp4   
4     9607838     videos_1000\9607838.mp4   

                                         description  
0                          Aerial shot winter forest  
1  Senior couple looking through binoculars on sa...  
2  A beautiful cookie with oranges lies on a gree...  
3  Japanese highrise office skyscrapers tokyo square  
4  Zrenjanin,serbia march 21 2015: fans watching ...  


In [6]:
df.head()

,video_id,video_path,description
0,21179416,videos_1000\21179416.mp4,Aerial shot winter forest
1,5629184,videos_1000\5629184.mp4,Senior couple looking through binoculars on sa...
2,1063125190,videos_1000\1063125190.mp4,A beautiful cookie with oranges lies on a gree...
3,1039695998,videos_1000\1039695998.mp4,Japanese highrise office skyscrapers tokyo square
4,9607838,videos_1000\9607838.mp4,"Zrenjanin,serbia march 21 2015: fans watching ..."


In [7]:
print(df['video_path'])

0        videos_1000\21179416.mp4
1         videos_1000\5629184.mp4
2      videos_1000\1063125190.mp4
3      videos_1000\1039695998.mp4
4         videos_1000\9607838.mp4
                  ...            
995      videos_1000\21687451.mp4
996       videos_1000\6260558.mp4
997      videos_1000\32789644.mp4
998      videos_1000\16009753.mp4
999    videos_1000\1034894669.mp4
Name: video_path, Length: 1000, dtype: object


In [8]:
# Function to extract frames from video
def extract_frames(video_path, num_frames=16):
    frames = []
    if not os.path.exists(video_path):
        print(f"Error: Video file does not exist {video_path}")
        return None
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return None
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = max(total_frames // num_frames, 1)
    for i in range(0, total_frames, interval):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(cv2.resize(frame, (224, 224)))
        if len(frames) == num_frames:
            break
    cap.release()
    if len(frames) < num_frames:
        print(f"Warning: Video {video_path} has fewer frames than expected ({len(frames)} out of {num_frames})")
        return None
    return frames

In [9]:
# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

# Reduce the training and validation sets to 50 samples each
train_df = train_df[:50]
val_df = val_df[:50]

# Prepare datasets
def prepare_dataset(df):
    data = []
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Preparing dataset"):
        #video_path = os.path.join(base_video_path, row['video_path'].replace("\\", "/"))
        video_path = os.path.join(row['video_path'].replace("\\", "/"))

        description = row['description']
        print(f"Processing video {idx}: {video_path}")
        frames = extract_frames(video_path)
        if frames:
            data.append((frames, description))
        else:
            print(f"Skipping video {idx} due to insufficient frames or read error: {video_path}")
    return data

train_data = prepare_dataset(train_df)
val_data = prepare_dataset(val_df)

Preparing dataset:   0%|          | 0/50 [00:00<?, ?it/s]

Processing video 716: videos_1000/1042107847.mp4


Preparing dataset:   4%|▍         | 2/50 [00:00<00:10,  4.48it/s]

Processing video 351: videos_1000/11387093.mp4
Processing video 936: videos_1000/22608862.mp4


Preparing dataset:   8%|▊         | 4/50 [00:00<00:08,  5.70it/s]

...

Preparing dataset:  96%|█████████▌| 48/50 [00:09<00:00,  4.28it/s]

Processing video 601: videos_1000/1014036899.mp4
Processing video 439: videos_1000/30106516.mp4


Preparing dataset:  98%|█████████▊| 49/50 [00:09<00:00,  4.98it/s]

Processing video 837: videos_1000/1033359257.mp4


Preparing dataset: 100%|██████████| 50/50 [00:10<00:00,  4.88it/s]


In [10]:
print(train_df)

       video_id                  video_path  \
716  1042107847  videos_1000\1042107847.mp4   
351    11387093    videos_1000\11387093.mp4   
936    22608862    videos_1000\22608862.mp4   
256  1024270121  videos_1000\1024270121.mp4   
635     3936698     videos_1000\3936698.mp4   
644  1017000745  videos_1000\1017000745.mp4   
554  1062850975  videos_1000\1062850975.mp4   
959    32921860    videos_1000\32921860.mp4   
168  1021233778  videos_1000\1021233778.mp4   
917  1064456023  videos_1000\1064456023.mp4   
528    12757697    videos_1000\12757697.mp4   
823     8394016     videos_1000\8394016.mp4   
985    12312911    videos_1000\12312911.mp4   
816  1024237247  videos_1000\1024237247.mp4   
86     30187849    videos_1000\30187849.mp4   
432  1012468937  videos_1000\1012468937.mp4   
184  1023817294  videos_1000\1023817294.mp4   
978    18328900    videos_1000\18328900.mp4   
534  1035272321  videos_1000\1035272321.mp4   
294  1008799799  videos_1000\1008799799.mp4   
892    173926

In [11]:
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_scheduler

# Load pre-trained model and components
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")


C:\Users\Abhishek Singh\AppData\Roaming\Python\Python312\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [19]:
class VideoCaptionDataset(Dataset):
    def __init__(self, data, feature_extractor, tokenizer):
        self.data = data
        self.feature_extractor = feature_extractor
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        frames, caption = self.data[idx]
        inputs = self.feature_extractor(images=frames, return_tensors="pt")
        inputs = {k: v for k, v in inputs.items()}  # Keep the batch dimension
        #print(f"Shape of pixel_values in __getitem__: {inputs['pixel_values'].shape}")  # Debug print

        target = self.tokenizer(caption, return_tensors="pt").input_ids.squeeze()
        return inputs, target


In [20]:
from torch.nn.utils.rnn import pad_sequence
# Custom collate function
def collate_fn(batch):
    # Stack pixel values (frames)
    pixel_values = torch.stack([item[0]['pixel_values'] for item in batch])
    #print(f"Shape of pixel_values in collate_fn: {pixel_values.shape}")  # Debug print

    # Pad the target sequences to the same length
    targets = pad_sequence([item[1] for item in batch], batch_first=True)
   # print(f"Shape of targets in collate_fn: {targets.shape}")  # Debug print

    return {'pixel_values': pixel_values}, targets

batch_size=1

# Create datasets and dataloaders
train_dataset = VideoCaptionDataset(train_data, feature_extractor, tokenizer)
val_dataset = VideoCaptionDataset(val_data, feature_extractor, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [21]:
# Initialize the model
model.train()


VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_featur

#Fine tuning

In [22]:
# Training parameters
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 2
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_featur

In [23]:
from torch.cuda.amp import GradScaler, autocast
scaler = GradScaler()
accumulation_steps = 1

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    for i, batch in enumerate(tqdm(train_loader, desc=f"Training epoch {epoch+1}/{num_epochs}")):
        inputs, targets = batch
        inputs = {k: v.to(device) for k, v in inputs.items()}
        targets = targets.to(device)

        batch_size, num_frames, channels, height, width = inputs['pixel_values'].shape
        flattened_pixel_values = inputs['pixel_values'].view(-1, channels, height, width)
        flattened_targets = targets.repeat_interleave(num_frames, dim=0)

        with autocast():
            outputs = model(pixel_values=flattened_pixel_values, labels=flattened_targets)
            loss = outputs.loss / accumulation_steps

        scaler.scale(loss).backward()

        if (i + 1) % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            torch.cuda.empty_cache()  # Clear CUDA cache to free up memory

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            inputs, targets = batch
            inputs = {k: v.to(device) for k, v in inputs.items()}
            targets = targets.to(device)

            flattened_pixel_values = inputs['pixel_values'].view(-1, channels, height, width)
            flattened_targets = targets.repeat_interleave(num_frames, dim=0)
            with autocast():
                outputs = model(pixel_values=flattened_pixel_values, labels=flattened_targets)
            val_loss += outputs.loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {avg_val_loss:.4f}")

Training epoch 1/2: 100%|██████████| 50/50 [15:23<00:00, 18.46s/it]


Epoch 1/2, Validation Loss: 5.1839


Training epoch 2/2: 100%|██████████| 50/50 [12:42<00:00, 15.25s/it]


Epoch 2/2, Validation Loss: 5.4581


In [29]:
# Save the fine-tuned model

# Save the fine-tuned model
model_save_path = 'copy_fine_tuned_vit_gpt2'
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)
feature_extractor.save_pretrained(model_save_path)


#tokenizer.save_pretrained('C:/Users/Abhishek Singh/Music/aip/MovieData/fine_tuned_vit_gpt2')


['copy_fine_tuned_vit_gpt2\\preprocessor_config.json']

In [30]:
torch.save(model.state_dict(), 'copyfine_tuned_gpt2.pth')


In [ ]:
#test case 

In [ ]:
# Function to generate captions
def generate_caption(model, feature_extractor, tokenizer, frames):
    inputs = feature_extractor(images=frames, return_tensors="pt")
    pixel_values = inputs.pixel_values
    output_ids = model.generate(pixel_values)
    caption = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return caption

In [ ]:
# Function to generate caption for a specific video
def generate_caption_for_video(video_path, model, feature_extractor, tokenizer):
    frames = extract_frames(video_path)
    caption = generate_caption(model, feature_extractor, tokenizer, frames)
    return caption

# Example usage:
video_path = r'C:\Users\Abhishek Singh\Music\aip\MovieData\test\ch.mp4'  
caption = generate_caption_for_video(video_path, model, feature_extractor, tokenizer)
print(f"Generated Caption: {caption}")

Generated Caption: a flock of white and black birds standing around 
